In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import tensorflow as tf

In [2]:
# settings
LEARNING_RATE = 1e-4
TRAINING_ITERATIONS = 2500  # set to 20000 on local environment to get 0.99 accuracy    
DROPOUT = 0.5
BATCH_SIZE = 50
VALIDATION_SIZE = 2000

In [3]:
# read training data from CSV file 
data = pd.read_csv('/Users/diniu/Desktop/Machine learning/Dtrain.csv')

print(data.shape)
print(data.head())

(42000, 785)
   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0      

In [4]:
images = data.iloc[:,1:].values
images = images.astype(np.float)

# convert from [0:255] => [0.0:1.0]
images = np.multiply(images, 1.0 / 255.0)

print(images.shape)

(42000, 784)


In [5]:
image_size = images.shape[1]
print('image_size:', image_size)

image_width = image_height = np.ceil(np.sqrt(image_size)).astype(np.uint8)
print('image_width:', image_width)
print('image_height:', image_height)

image_size: 784
image_width: 28
image_height: 28


In [6]:
labels_flat = data[[0]].values.ravel()
print(labels_flat.shape)

(42000,)


In [7]:
labels_count = np.unique(labels_flat).shape[0]
print(labels_count)

10


In [8]:
# transform labels_flat to one-hot
def dense_to_one_hot(labels_dense, num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

labels = dense_to_one_hot(labels_flat, labels_count)
labels = labels.astype(np.uint8)
print(labels.shape)

(42000, 10)


In [9]:
# split data into training & validation
validation_images = images[:VALIDATION_SIZE]
validation_labels = labels[:VALIDATION_SIZE]

train_images = images[VALIDATION_SIZE:]
train_labels = labels[VALIDATION_SIZE:]

print('train images size:', train_images.shape)
print('train labels size:', train_labels.shape)
print('validation images size:', validation_images.shape)
print('validation labels size:', validation_labels.shape)

train images size: (40000, 784)
train labels size: (40000, 10)
validation images size: (2000, 784)
validation labels size: (2000, 10)


In [10]:
# weight initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [11]:
# convolution with padding such that input and output have the same 'image size'
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [12]:
# pooling - from 2n*2n to n*n
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [13]:
# input & output of NN
x = tf.placeholder('float', shape=[None, image_size])
y_ = tf.placeholder('float', shape=[None, labels_count])

In [14]:
# first convolutional layer - window size: 5*5 - 32 filters
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# (40000,784) => (40000,28,28,1)
image = tf.reshape(x, [-1,image_width , image_height,1])

# activate function - ReLU
h_conv1 = tf.nn.relu(conv2d(image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [15]:
# second convolutional layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
#print (h_conv2.get_shape()) # => (40000, 14,14, 64)
h_pool2 = max_pool_2x2(h_conv2)
#print (h_pool2.get_shape()) # => (40000, 7, 7, 64)

In [16]:
# densely connected layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# (40000, 7, 7, 64) => (40000, 3136)
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

# (40000, 1024)
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [17]:
# dropout when training - prevent overfitting
keep_prob = tf.placeholder('float')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [18]:
# readout layer for deep net
W_fc2 = weight_variable([1024, labels_count])
b_fc2 = bias_variable([labels_count])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [19]:
# cost function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

# optimisation function
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)

# evaluation
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [20]:
# prediction
predict = tf.argmax(y_conv,1)

In [21]:
# serve data by batches
epochs_completed = 0
index_in_epoch = 0
num_examples = train_images.shape[0]

def next_batch(batch_size):
    
    global train_images
    global train_labels
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    # when all trainig data have been already used, it is reorder randomly    
    if index_in_epoch > num_examples:
        # finished epoch
        epochs_completed += 1
        print('The number of completed epochs is', epochs_completed)
        # shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        train_images = train_images[perm]
        train_labels = train_labels[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return train_images[start:end], train_labels[start:end]

In [22]:
# start TensorFlow session
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()

sess.run(init)

In [23]:
# useful variables for log
train_accuracies = []
validation_accuracies = []
x_range = []

display_step=1

for i in range(TRAINING_ITERATIONS):

    #get new batch
    batch_xs, batch_ys = next_batch(BATCH_SIZE)        

    # check evaluation on every 1st,2nd,...,10th,20th,...,100th... step
    if i%display_step == 0 or (i+1) == TRAINING_ITERATIONS:
        
        train_accuracy = accuracy.eval(feed_dict={x: batch_xs, 
                                                  y_: batch_ys, 
                                                  keep_prob: 1.0})       
        if(VALIDATION_SIZE):
            validation_accuracy = accuracy.eval(feed_dict={ x: validation_images[0:BATCH_SIZE], 
                                                            y_: validation_labels[0:BATCH_SIZE], 
                                                            keep_prob: 1.0})                                  
            print('training_accuracy / validation_accuracy => %.2f / %.2f for step %d'%(train_accuracy, validation_accuracy, i))
            
            validation_accuracies.append(validation_accuracy)
            
        else:
             print('training_accuracy => %.4f for step %d'%(train_accuracy, i))
        train_accuracies.append(train_accuracy)
        x_range.append(i)
        
        # increase display_step
        if i%(display_step*10) == 0 and i:
            display_step *= 10
    
    # train on batch
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: DROPOUT})

training_accuracy / validation_accuracy => 0.08 / 0.08 for step 0
training_accuracy / validation_accuracy => 0.12 / 0.16 for step 1
training_accuracy / validation_accuracy => 0.16 / 0.14 for step 2
training_accuracy / validation_accuracy => 0.16 / 0.26 for step 3
training_accuracy / validation_accuracy => 0.28 / 0.28 for step 4
training_accuracy / validation_accuracy => 0.28 / 0.30 for step 5
training_accuracy / validation_accuracy => 0.24 / 0.28 for step 6
training_accuracy / validation_accuracy => 0.20 / 0.30 for step 7
training_accuracy / validation_accuracy => 0.14 / 0.30 for step 8
training_accuracy / validation_accuracy => 0.28 / 0.30 for step 9
training_accuracy / validation_accuracy => 0.40 / 0.30 for step 10
training_accuracy / validation_accuracy => 0.46 / 0.40 for step 20
training_accuracy / validation_accuracy => 0.54 / 0.60 for step 30
training_accuracy / validation_accuracy => 0.70 / 0.64 for step 40
training_accuracy / validation_accuracy => 0.62 / 0.80 for step 50
train

In [24]:
# check final accuracy on full validation set  
if(VALIDATION_SIZE):
    validation_accuracy = accuracy.eval(feed_dict={x: validation_images, 
                                                   y_: validation_labels, 
                                                   keep_prob: 1.0})
    print('validation_accuracy: ', validation_accuracy)

validation_accuracy:  0.9835


In [26]:
# read test data from CSV file 
test_images = pd.read_csv('/Users/diniu/Desktop/Machine learning/Dtest.csv').values
test_images = test_images.astype(np.float)

# convert from [0:255] => [0.0:1.0]
test_images = np.multiply(test_images, 1.0 / 255.0)

print('test images size: ', test_images.shape)

test images size:  (28000, 784)


In [27]:
# predict test set using batches
predicted_lables = np.zeros(test_images.shape[0])
for i in range(0,test_images.shape[0]//BATCH_SIZE):
    predicted_lables[i*BATCH_SIZE : (i+1)*BATCH_SIZE] = predict.eval(feed_dict={x: test_images[i*BATCH_SIZE : (i+1)*BATCH_SIZE], 
                                                                                keep_prob: 1.0})

print('predicted lables lenth: ', len(predicted_lables))

predicted lables lenth:  28000


In [29]:
# save to submission
np.savetxt('submission.csv', 
           np.c_[range(1,len(test_images)+1),predicted_lables], 
           delimiter=',', 
           header = 'ImageId,Label', 
           comments = '', 
           fmt='%d')